In [7]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service

service = Service('/Users/skg/Desktop/test/chromedriver')
driver = webdriver.Chrome(service=service)

In [2]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium import webdriver
import time
import pandas as pd
import json
#放chromedriver的文件路径加文件名
CHROME_DRIVER = '/Users/skg/Desktop/test/chromedriver'


In [14]:
'''# 创建chrome参数对象，这一段可以根据自己需求百度，尽量提高爬虫速度
opt = webdriver.ChromeOptions()
opt.add_argument('blink-settings=imagesEnabled=false') #不加载图片, 提升速度
options=opt)'''
#设置浏览器参数
driver = webdriver.Chrome()
#driver = webdriver.Chrome(executable_path=CHROME_DRIVER)

options = webdriver.ChromeOptions()
options.add_argument('--no-sandbox')
options.add_argument('--disable-dev-shm-usage')



SessionNotCreatedException: Message: session not created: This version of ChromeDriver only supports Chrome version 86
Current browser version is 115.0.5790.114 with binary path /Applications/Google Chrome.app/Contents/MacOS/Google Chrome
Stacktrace:
0   chromedriver                        0x000000010895ac29 chromedriver + 2464809
1   chromedriver                        0x0000000108f956f3 chromedriver + 8996595
2   chromedriver                        0x000000010879fd56 chromedriver + 650582
3   chromedriver                        0x0000000108719f9a chromedriver + 102298
4   chromedriver                        0x0000000108716177 chromedriver + 86391
5   chromedriver                        0x0000000108713430 chromedriver + 74800
6   chromedriver                        0x0000000108745326 chromedriver + 279334
7   chromedriver                        0x00000001087423f3 chromedriver + 267251
8   chromedriver                        0x000000010871c230 chromedriver + 111152
9   chromedriver    

In [ ]:
#href文件是要爬取的链接库，如果只需要爬取一个文件，只需要设置href为网址
href=pd.read_csv('mer_info.csv',encoding='GBK')
href = href ['href']
driver.get(href[0])


#打开网站后手动登陆一下大众点评，再执行下面的语句


In [8]:
#获取cookies
cookies = driver.get_cookies()
'''f1 = open('cookie.txt', 'w')
f1.write(json.dumps(cookies))
f1.close
f1 = open('cookie.txt')
cookie = f1.read()
cookie =json.loads(cookie)'''

NameError: name 'driver' is not defined

In [ ]:
#加载cookie，之后载入的界面会自动登陆
driver.add_cookie(cookies[0])
#抓取标签，自定义函数x是返回的list，y是内容的xpath路径，z是需要get的属性
def get_tag(x,y,z):
    xpath = y
    doc = driver.find_elements_by_xpath(xpath)
    x=[c.get_attribute(z) for c in doc]
    #x = x+b
    return x

#定义一些空列表
#list(range(0,len(href))

user_id = []
comment_time = []
comennt1 = []
comment2=[]

#更改第下面的range里面的参数，100-200，200-300，300-400，400-500，500-600，600-len(href)
for i in list(range(0,len(href)):
    #载入网址,大众点评的网址还是很规律的，除了主页网址，商户地址就是shop/商户号，用户地址就是 
    #member/用户号，搜索就是搜索字符的转码
    url = href[i] + '/review_all'
    driver.get(url)
    
    #评论页面循环
    for m in list(range(0,100)):
        #选取特定时间评论的内容
        comment_time = driver.find_elements_by_class_name('time')
        comment_time = [n.text.split(' ')[0] for n in comment_time]
        x = pd.DataFrame({'time':comment_time})
        x['time']=pd.to_datetime(x['time'],format = '%Y-%m-%d %H:%M')

        #如果该页评论时间中，在2018年7月27日之后评论的数量少于2个，就不抓取
        if sum(x['time'] > pd.to_datetime('2018-7-27 15:00',format = '%Y-%m-%d %H:%M'))<=2:
            break
        else:
            #抓取评论用户的id
            user_id = get_tag(user_id,'//*[@id="reviewlist"]/div[2]/div[1]/div[3]/div[3]/ul/li/div/div[1]/a','href')
            #自动点开展开评论
            for j in list(range(1,25)):
                try:
                    driver.find_element_by_link_text('展开评论').click()
                except:
                    break
            

            #抓取用户评论的字数，如果要评论只需要把len函数去掉即可，但是大众点评随机把文字替换成了图片，抓取的评论会有很多的缺失
            comment1 = driver.find_elements_by_xpath('//*[@id="review-list"]/div[2]/div[1]/div[3]/div[3]/ul/li/div/div[3]')
            comment2 = driver.find_elements_by_xpath('//*[@id="review-list"]/div[2]/div[1]/div[3]/div[3]/ul/li/div/div[4]')

            if len(comment2)==0:
                comment1 = driver.find_elements_by_xpath('//*[@id="review-list"]/div[2]/div[1]/div[3]/div[2]/ul/li/div/div[3]')
                comment2 = driver.find_elements_by_xpath('//*[@id="review-list"]/div[2]/div[1]/div[3]/div[2]/ul/li/div/div[4]')
            else:
                pass
            comment_len1 = [len(k.text) for k in comment1]
            comment_len2 = [len(k.text) for k in comment2]
            try:
                #数据拼接成数据框
                y = pd.DataFrame({'usre_id':user_id,
                            'time':x['time'],
                            'comment_len1':comment_len1,
                            'comment_len2':comment_len2})
                y['mer_href'] = href[i]

############下面这个i一定要改成和上面起始值一样的，不然会报错
            if i == 0 &m==0:
                mer_comment_info_0 = y
            else:
                mer_comment_info_0 = pd.concat([mer_comment_info_0,y],axis = 0)
                mer_comment_info_0.to_csv('D:\\mer_comment_info156-667.csv')
                #没加入一页评论，输出一个字符，随时查看进度
                print(str(i)+'read'+str(m))   
            except:
                #如果发生错误不要停止程序，同时输出错误的页码
                print(str(i)+'!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!'+str(m))
                pass
                
        try:
            #翻页
            driver.find_element_by_link_text('下一页').click()
        except:
            #如果最后一页，进入下一个href
            break
